In [1]:
# Install dependencies
!pip install roboflow
!pip install ultralytics
!pip install opencv-python


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.9/80.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.3/887.3 kB 13.2 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install pydub
!apt-get install ffmpeg -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [ ]:
# Import required librarie

from roboflow import Roboflow
rf = Roboflow(api_key="xPdNpfiFR6lHiwvO7kdv")
project = rf.workspace("miniproject-hjhth").project("smart_rail_system")
version = project.version(10)
dataset = version.download("yolov8")





# Output the downloaded dataset location
print(dataset)




In [ ]:
import os
os.environ['WANDB_MODE'] = 'offline'


In [ ]:
# Import YOLOv8
from ultralytics import YOLO

# Load a YOLOv8 model
model = YOLO("yolov8n.pt")  # or yolov8s.pt or yolov8m.pt depending on your preference

# Train the YOLOv8 model
results = model.train(
    data="/content/smart_rail_system-10/data.yaml",  # Correct dataset path
    epochs=20,  # Number of epochs for training
    imgsz=640   # Image size
)


In [ ]:
# Validate the model
metrics = model.val(data="/content/smart_rail_system-10/data.yaml")  # Validation on the dataset
print(metrics.box.map)  # Output the mean Average Precision for bounding boxes


In [ ]:
# Load the trained model for inference
#model = YOLO('/content/best.pt')  # Path to the best model after training

# Inference on a new image
results = model.predict(source="/content/1.jpg", save=True)


In [2]:
# Import necessary libraries
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
import cv2
# Specific Colab import for displaying images
from google.colab.patches import cv2_imshow  # Import cv2_imshow
# JavaScript function to start the webcam and capture frames
def video_stream():
    js = Javascript('''
        var video;
        var div = null;
        var stream;
        var captureCanvas;

        async function startVideo() {
          if (div !== null) {
            return { captureFrame: captureFrame };
          }

          div = document.createElement('div');
          div.style.border = '2px solid black';
          div.style.padding = '3px';
          div.style.width = '100%';
          div.style.maxWidth = '600px';
          document.body.appendChild(div);

          video = document.createElement('video');
          video.style.display = 'block';
          video.width = div.clientWidth - 6;
          video.setAttribute('playsinline', '');
          stream = await navigator.mediaDevices.getUserMedia({video: { facingMode: "environment" }});
          div.appendChild(video);

          video.srcObject = stream;
          await video.play();

          captureCanvas = document.createElement('canvas');
          captureCanvas.width = 640;
          captureCanvas.height = 480;

          function captureFrame() {
            captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
            return captureCanvas.toDataURL('image/jpeg', 0.8);
          }

          return { captureFrame: captureFrame };
        }

        async function stream_frame() {
          const videoControl = await startVideo();
          return videoControl.captureFrame();
        }
    ''')
    display(js)


In [3]:
# Function to convert the JavaScript-captured frame into an OpenCV image
def js_to_image(js_reply):
    img_bytes = b64decode(js_reply.split(',')[1])  # Decode the image data
    img_array = np.frombuffer(img_bytes, dtype=np.uint8)
    img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
    return img


In [4]:
def overlap(box1, box2):
    x1, y1, x2, y2 = box1
    gx1, gy1, gx2, gy2 = box2

    # Check if there is overlap
    if x1 < gx2 and x2 > gx1 and y1 < gy2 and y2 > gy1:
        return True
    return False






def draw_bboxes(frame, filtered_boxes):
    color = (255, 255, 255)
    humans = []
    gestures = []
    gestures2 = []
    for box in filtered_boxes:
        # Get bounding box coordinates
        x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
        xyxy = box.xyxy[0].cpu().numpy().astype(int)
        conf = box.conf[0].item()  # Get confidence score as a float
        cls = int(box.cls[0])  # Get class index as an integer

        # Set color and label based on class
        if cls == 0:  # Crime detection
            label = "crime"
            color = (0, 0, 255)  # Red for crime
            gestures2.append((x1, y1, x2, y2))
        elif cls == 1:  # Emergency alert
            label = "emergency"
            color = (0, 255, 255)  # Yellow for emergency
            gestures.append((x1, y1, x2, y2))
        elif cls == 2:  # Human detection
            label = "human"
            color = (255, 0, 0)  # Blue for human
            humans.append((x1, y1, x2, y2))

        for human_box in humans:
          for gesture_box in gestures:
            if overlap(human_box, gesture_box):
                # If human and gesture overlap, draw a bounding box around the human
                x1, y1, x2, y2 = human_box
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
                cv2.putText(frame, 'EMERGENCY', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        for human_box in humans:
          for gesture_box in gestures2:
            if overlap(human_box, gesture_box):
                # If human and gesture overlap, draw a bounding box around the human
                x1, y1, x2, y2 = human_box
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
                cv2.putText(frame, 'CRIME INCIDENT', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

        # Draw bounding box and label on the frame
        #cv2.rectangle(frame, (xyxy[0], xyxy[1]), (xyxy[2], xyxy[3]), color, 2)
        #cv2.putText(frame, f"{label} {conf:.2f}", (xyxy[0], xyxy[1] - 10),cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    return frame

In [5]:
def draw_bboxes(frame, filtered_boxes):
    color = (255, 255, 255)
    for box in filtered_boxes:
        # Get bounding box coordinates
        xyxy = box.xyxy[0].cpu().numpy().astype(int)
        conf = box.conf[0].item()  # Get confidence score as a float
        cls = int(box.cls[0])  # Get class index as an integer

        # Set color and label based on class
        if cls == 0:  # Crime detection
            label = "crime"
            color = (0, 0, 255)  # Red for crime
        elif cls == 1:  # Emergency alert
            label = "emergency"
            color = (0, 255, 255)  # Yellow for emergency
        elif cls == 2:  # Human detection
            label = "human"
            color = (255, 0, 0)  # Blue for human

        # Draw bounding box and label on the frame
        cv2.rectangle(frame, (xyxy[0], xyxy[1]), (xyxy[2], xyxy[3]), color, 2)
        cv2.putText(frame, f"{label} {conf:.2f}", (xyxy[0], xyxy[1] - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    return frame

In [6]:
import time
from IPython.display import Audio, display, Javascript

display(Javascript("""
    (async () => {
        const audio = document.querySelector('audio');
        if (audio) {
            await audio.play();
        }
    })();
"""))

<IPython.core.display.Javascript object>

In [12]:
import cv2
import time
from ultralytics import YOLO
from IPython.display import Audio, display
import warnings

# Suppress unnecessary YOLO logging
warnings.filterwarnings('ignore')

# Define confidence thresholds for each class
CONF_THRESH = {
    0: 0.5,  # Human detection confidence threshold
    1: 0.5,  # Emergency detection confidence threshold
    2: 0.7   # Crime detection confidence threshold
}

# Load your trained YOLOv8 model
model = YOLO('/content/best.pt')  # Load your model

# Start the webcam stream and continuously detect objects
video_stream()  # Start the video stream

frame_count = 0  # Initialize frame counter
max_frames = 100  # Limit the number of frames to process

# Continuous detection loop
try:
    while frame_count < max_frames:  # Stop after processing max_frames
        js_reply = eval_js('stream_frame()')  # Capture frames continuously from the webcam
        if not js_reply:
            break

        # Convert the JS frame to an OpenCV image
        frame = js_to_image(js_reply)

        # Run YOLOv8 detection on the captured frame
        results = model(frame, verbose=False)  # Disable verbose output

        # Filter detections by confidence threshold for each class
        detected_classes = set()
        filtered_boxes = []
        for result in results:
            for box in result.boxes:
                class_id = int(box.cls[0])
                confidence = float(box.conf[0])

                # Apply confidence threshold filtering
                if confidence >= CONF_THRESH.get(class_id, 0):
                    detected_classes.add(class_id)
                    filtered_boxes.append(box)

        # Draw bounding boxes if conditions are met
        if 2 in detected_classes and 0 in detected_classes:
            # Draw bounding boxes on the frame with filtered results
            frame_with_bboxes = draw_bboxes(frame, filtered_boxes)
            cv2.imwrite(f"/content/frame_{frame_count}.jpg", frame_with_bboxes)
            print("CRIME DETECTED!!! - "+f"REPORT CAPTURED AND SAVED AS : Frame {frame_count} .")
            display(Audio('/content/ElevenLabs_Text_to_Speech_audio_crime.mp3', autoplay=True))
            time.sleep(4)

        if 2 in detected_classes and 1 in detected_classes:
            # Draw bounding boxes on the frame with filtered results
            frame_with_bboxes = draw_bboxes(frame, filtered_boxes)
            cv2.imwrite(f"/content/frame_{frame_count}.jpg", frame_with_bboxes)
            print("EMERGENCY DETECTED!!! - "+f"REPORT CAPTURED AND SAVED AS : Frame {frame_count} .")
            display(Audio('/content/ElevenLabs_Text_to_Speech_audio11.mp3', autoplay=True))
            time.sleep(4)

        # Increment the frame counter
        frame_count += 1

    print("Finished processing frames.")

except KeyboardInterrupt:
    print("Interrupted by user")

except Exception as e:
    print(f"An error occurred: {str(e)}")



<IPython.core.display.Javascript object>

EMERGENCY DETECTED!!! - REPORT CAPTURED AND SAVED AS : Frame 3 .


EMERGENCY DETECTED!!! - REPORT CAPTURED AND SAVED AS : Frame 7 .


EMERGENCY DETECTED!!! - REPORT CAPTURED AND SAVED AS : Frame 8 .


CRIME DETECTED!!! - REPORT CAPTURED AND SAVED AS : Frame 10 .


Interrupted by user
